In [ ]:
import os, sys
import gspread, httplib2
from oauth2client.client import OAuth2WebServerFlow
from oauth2client import file, tools
from google_ids import *

filename = "oauth.dat"


class Unbuffered(object):
    def __init__(self, stream):
        self.stream = stream
    def write(self, data):
        self.stream.write(data)
        self.stream.flush()
    def __getattr__(self, attr):
        return getattr(self.stream, attr)


class MyCreds (object): 
    def __init__ (self, access_token=None): 
        self.access_token = access_token 


def get_email():
    """May end up moving this into GoodSheet"""
    service = create_google_service(filename=filename, api_name="oauth2", version="v2")
    user_document = service.userinfo().get().execute()
    email = user_document['email']
    return email


def create_google_service(filename, api_name, version):
    """This lets you create instances of Google Services."""
    from apiclient.discovery import build
    path = os.path.dirname(os.path.realpath('__file__'))
    path_filename = os.path.join(path, filename)
    storage = file.Storage(path_filename)
    credentials = storage.get()
    http = credentials.authorize(http = httplib2.Http())
    service = build(api_name, version, http)
    return service


def oauth():
    import argparse, json
    scopes = ["https://www.googleapis.com/auth/analytics.readonly", 
              "https://www.googleapis.com/auth/webmasters.readonly",
              "https://www.googleapis.com/auth/yt-analytics.readonly",
              "https://www.googleapis.com/auth/youtube.readonly",
              "https://spreadsheets.google.com/feeds/",
              "https://www.googleapis.com/auth/gmail.modify",
              "https://www.googleapis.com/auth/userinfo.email"]
    
    path = os.path.dirname(os.path.realpath('__file__'))
    path_filename = os.path.join(path, filename)
    flow = OAuth2WebServerFlow(client_id, client_secret, scopes, 
                               redirect_uri='urn:ietf:wg:oauth:2.0:oob',
                               response_type='code',
                               prompt='consent',
                               access_type='offline')
    authorize_url = flow.step1_get_authorize_url()
    storage = file.Storage(path_filename)
    credentials = storage.get()
    argparser = argparse.ArgumentParser(add_help=False)
    parents = [argparser]
    parent_parsers = [tools.argparser]
    parent_parsers.extend(parents)
    parser = argparse.ArgumentParser(
        description="__doc__",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        parents=parent_parsers)
    flags = parser.parse_args(['--noauth_local_webserver'])
    try:
        http = credentials.authorize(http = httplib2.Http())
    except:
        pass
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(flow, storage, flags)
    else:
        credentials.refresh(http)
    with open(path_filename) as json_file:
        jdata = json.load(json_file)
    token = jdata['access_token']
    creds = MyCreds(access_token=token)
    connect = gspread.authorize(creds)
    return connect

def proxy_generator(num_pages):
    import hma_scraper
    from random import shuffle
    from collections import namedtuple
    from urllib.parse import urlparse
    import requests
    from requests.packages.urllib3.exceptions import InsecureRequestWarning
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    Proxy = namedtuple('Proxy', 'good scheme address seconds')
    proxy_list = hma_scraper.scrape_hma(num_pages)
    shuffle(proxy_list)
    for a_proxy in proxy_list:
        url_parts = urlparse(a_proxy)
        requests_arg = {url_parts.scheme: a_proxy}
        try:
            result = requests.get('https://www.google.com', proxies=requests_arg, verify=False, timeout=1)
            yield Proxy(True, url_parts.scheme, a_proxy, result.elapsed.total_seconds())
        except:
            pass


# Generate a list of proxies for glboal standby use, for serps and stuff.
proxies = proxy_generator(2)
proxy = next(proxies)

# Force the 1st instance of OAuth2 authentication prompt or refresh.
force_it = oauth()

# Force IPython Notebook to not buffer output
sys.stdout = Unbuffered(sys.stdout)

In [ ]:
proxies = proxy_generator(5)